https://www.kaggle.com/code/msripooja/steps-to-convert-audio-clip-to-spectrogram

In [113]:
import warnings

warnings.filterwarnings('ignore')

In [114]:
from utils import *

In [115]:
X_train, y_train = generate_dataset(directory='./TrainingData', MAX_LEN=170)

In [116]:
X_test, y_test = generate_dataset(directory='./TestingData', MAX_LEN=170)

In [117]:
X_train.shape

(120, 170, 33)

In [118]:
X_test.shape

(40, 170, 33)

In [119]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, accuracy_score

In [120]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [124]:
model = models.Sequential([
    layers.Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),  # Apply across time steps
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(64, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(64, activation='relu', kernel_regularizer='l2'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

In [125]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=4,  # Smaller batch size for small datasets
    validation_split=0.2,
    verbose=2
)

Epoch 1/20
24/24 - 1s - 31ms/step - accuracy: 0.5729 - loss: 1.6897 - val_accuracy: 0.3750 - val_loss: 1.1934
Epoch 2/20
24/24 - 0s - 3ms/step - accuracy: 0.5104 - loss: 1.1007 - val_accuracy: 0.3333 - val_loss: 1.0605
Epoch 3/20
24/24 - 0s - 3ms/step - accuracy: 0.6042 - loss: 0.9701 - val_accuracy: 0.3333 - val_loss: 1.0450
Epoch 4/20
24/24 - 0s - 3ms/step - accuracy: 0.5417 - loss: 0.9364 - val_accuracy: 0.4167 - val_loss: 0.9432
Epoch 5/20
24/24 - 0s - 4ms/step - accuracy: 0.6979 - loss: 0.8387 - val_accuracy: 0.3333 - val_loss: 1.0881
Epoch 6/20
24/24 - 0s - 4ms/step - accuracy: 0.6875 - loss: 0.8378 - val_accuracy: 0.3750 - val_loss: 0.9321
Epoch 7/20
24/24 - 0s - 4ms/step - accuracy: 0.6146 - loss: 0.7426 - val_accuracy: 0.5000 - val_loss: 0.9119
Epoch 8/20
24/24 - 0s - 4ms/step - accuracy: 0.7604 - loss: 0.6904 - val_accuracy: 0.6250 - val_loss: 0.8661
Epoch 9/20
24/24 - 0s - 4ms/step - accuracy: 0.8125 - loss: 0.6208 - val_accuracy: 0.5833 - val_loss: 0.8125
Epoch 10/20
24/24 

In [126]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy:.2f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.7937 - loss: 0.5952 
Test Accuracy: 0.80


In [ ]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int) 

print("\nClassification Report:\n", classification_report(y_test, y_pred_binary))

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step

Classification Report:
               precision    recall  f1-score   support

           0       0.83      0.75      0.79        20
           1       0.77      0.85      0.81        20

    accuracy                           0.80        40
   macro avg       0.80      0.80      0.80        40
weighted avg       0.80      0.80      0.80        40

